In [1]:
%load_ext nb_black
# !pip install nb_black

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn import metrics as m
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE  # doctest: +NORMALIZE_WHITESPACE
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
import xgboost as xgb
from sklearn.pipeline import make_pipeline
from mlxtend.feature_selection import ColumnSelector
from sklearn import model_selection
from mlxtend.classifier import StackingClassifier
from thundersvm import SVC as svmgpu

import warnings

warnings.filterwarnings("ignore")
randomseed = 7
np.random.seed(randomseed)

Using TensorFlow backend.


<IPython.core.display.Javascript object>

# 1. Read the dataset

In [3]:
x_original = pd.read_csv("../dataset/XLable_onlyDiabeticRemoved.txt")

conditions = [
    (x_original.L100800 < 100),
    (x_original.L100800 >= 100) & (x_original.L100800 < 126),
    (x_original.L100800 >= 126),
]
choices = [0, 1, 2]
x_original["CLASS"] = np.select(conditions, choices, default=0)
x_original = x_original[
    [
        "Unnamed: 0",
        "L100800",
        "L104600",
        "L103000",
        "S000300",
        "L101700",
        "L100700",
        "FIELD_33",
        "FIELD_38",
        "FIELD_40",
        "FIELD_31",
        "SEX",
        "AGE",  #'CLASS',
        "FIELD_16",
        "FIELD_23",
        "FIELD_15",
        "FIELD_22",
        "FIELD_17",
        "FIELD_24",
    ]
]

print(x_original.shape)

(185843, 19)


<IPython.core.display.Javascript object>

In [4]:
y_original = pd.read_csv("../dataset/TargetLable_onlyDiabeticRemoved.txt")

conditions = [
    (y_original.L100800 < 100),
    (y_original.L100800 >= 100) & (y_original.L100800 < 126),
    (y_original.L100800 >= 126),
]

choices = [0, 1, 2]
y_original["CLASS"] = np.select(conditions, choices, default=0)

y_original = y_original[["Unnamed: 0", "CLASS"]]

print(y_original.shape)

(185843, 2)


<IPython.core.display.Javascript object>

In [5]:
data = pd.merge(
    x_original, y_original, how="inner", left_on="Unnamed: 0", right_on="Unnamed: 0"
)

<IPython.core.display.Javascript object>

In [6]:
# filter the data set
data = data[data.FIELD_16 != 1]  # exclude people who are diagnosed for (diabetes)
data = data[data.FIELD_23 != 1]  # exclude people who are on medication for diabetes

data = data[
    data.FIELD_15 != 1
]  # exclude people who are diagnosed for (high blood pressure)
data = data[
    data.FIELD_22 != 1
]  # exclude people who are on medication for high blood pressure

data = data[data.FIELD_17 != 1]  # exclude people who are diagnosed for hyperlipidemia
data = data[
    data.FIELD_24 != 1
]  # exclude people who are on medication for hyperlipidemia

print(data.shape)

(169024, 20)


<IPython.core.display.Javascript object>

In [7]:
data = data[
    [
        "L100800",
        "L104600",
        "L103000",
        "S000300",
        "L101700",
        "L100700",
        "FIELD_33",
        "FIELD_38",
        "FIELD_40",
        "FIELD_31",
        "SEX",
        "AGE",
        "CLASS",
    ]
]
data = data.dropna()
print(data.shape)
data.head()

(56542, 13)


,L100800,L104600,L103000,S000300,L101700,L100700,FIELD_33,FIELD_38,FIELD_40,FIELD_31,SEX,AGE,CLASS
2,78.0,5.28,41.0,20.2,15.0,3.8,1.0,2.0,1.0,0.0,1.0,46.0,0
5,90.0,5.74,50.0,25.5,12.0,3.4,1.0,0.0,1.0,0.0,1.0,52.0,0
10,86.0,5.83,45.0,21.2,17.0,3.9,1.0,0.0,1.0,1.0,1.0,37.0,0
11,86.0,4.73,54.0,22.0,30.0,4.2,1.0,2.0,3.0,0.0,1.0,39.0,0
20,87.0,5.60,340.0,24.6,26.0,4.7,1.0,0.0,2.0,0.0,1.0,59.0,0


<IPython.core.display.Javascript object>

# 2. Downsample the majority class and upsample the minority

In [8]:
diabetic = data[data.CLASS == 2]
prediabetic = data[data.CLASS == 1]
normal = data[data.CLASS == 0]

print(diabetic.shape[0], prediabetic.shape[0], normal.shape[0])

1045 17331 38166


<IPython.core.display.Javascript object>

In [9]:
diabetic_test = diabetic.sample(200, random_state=randomseed)
prediabetic_test = prediabetic.sample(200, random_state=randomseed)
normal_test = normal.sample(200, random_state=randomseed)
test = pd.concat([diabetic_test, prediabetic_test, normal_test])

diabetic_train = diabetic.drop(diabetic_test.index)
prediabetic_train = prediabetic.drop(prediabetic_test.index)
# .sample(
#     10 * diabetic_train.shape[0], random_state=randomseed
# )
normal_train = normal.drop(normal_test.index).sample(
    prediabetic_train.shape[0],
    random_state=randomseed
    #     10 * diabetic_train.shape[0], random_state=randomseed
)
train = pd.concat([diabetic_train, diabetic_train, prediabetic_train, normal_train])

<IPython.core.display.Javascript object>

In [10]:
xtrain = train.iloc[:, :-1]
ytrain = train.iloc[:, -1]
xtest = test.iloc[:, :-1]
ytest = test.iloc[:, -1]

<IPython.core.display.Javascript object>

In [11]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
xtrain = scaler.fit_transform(xtrain)
xtest = scaler.transform(xtest)

<IPython.core.display.Javascript object>

In [12]:
from imblearn.over_sampling import SMOTE, SMOTENC  # doctest: +NORMALIZE_WHITESPACE

randomseed = 42

sm = SMOTENC(
    random_state=randomseed,
    categorical_features=[6, 7, 8, 9, 10],
    sampling_strategy="minority",
)
X_res, y_res = sm.fit_resample(xtrain, ytrain)

print("Resampled dataset shape %s" % Counter(y_res))
print(
    y_res[y_res == 0].shape[0], y_res[y_res == 1].shape[0], y_res[y_res == 2].shape[0]
)
print(X_res.shape, y_res.shape)

xtrain = X_res
ytrain = y_res

Resampled dataset shape Counter({2: 17131, 1: 17131, 0: 17131})
17131 17131 17131
(51393, 12) (51393,)


<IPython.core.display.Javascript object>

# 3. Generate the classifier models based on the selected  features

# Models

In [13]:
# generate the models based on the selected columns list and the ml classifiers

weakmodles = []
estimators = []

<IPython.core.display.Javascript object>

# 3.1. 12 features

In [14]:
rf_model_12 = RandomForestClassifier(
    bootstrap=True,
    class_weight=None,
    criterion="gini",
    max_depth=10,
    max_features="log2",
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    min_samples_leaf=2,
    min_samples_split=12,
    min_weight_fraction_leaf=0.0,
    n_estimators=100,
    n_jobs=-1,
    oob_score=False,
    random_state=randomseed,
    verbose=1,
    warm_start=False,
)

xgb_model_12 = xgb.XGBClassifier(objective="binary:logistic", random_state=randomseed)

scv_model_12 = svmgpu(
    C=100,
    cache_size=100,
    class_weight={},
    coef0=0.0,
    decision_function_shape="ovo",
    degree=3,
    gamma=0.1,
    gpu_id=0,
    kernel="linear",
    max_iter=-1,
    max_mem_size=-1,
    n_jobs=-1,
    probability=False,
    random_state=randomseed,
    shrinking=False,
    tol=0.001,
    verbose=False,
)

scv_model_cpu_12 = SVC(
    C=100,
    cache_size=100,
    class_weight={},
    coef0=0.0,
    decision_function_shape="ovo",
    degree=3,
    gamma=0.1,
    #     gpu_id=0,
    kernel="linear",
    max_iter=-1,
    #     max_mem_size=-1,
    #     n_jobs=-1,
    probability=False,
    random_state=randomseed,
    shrinking=False,
    tol=0.001,
    verbose=False,
)

weakmodles.append(rf_model_12)
weakmodles.append(xgb_model_12)
weakmodles.append(scv_model_12)

estimators.append(("rf_model_12", rf_model_12))
estimators.append(("xgb_model_12", xgb_model_12))
estimators.append(("scv_model_12", scv_model_cpu_12))

# estimators = [
#     ("rf_model_12", rf_model_12),
#     ("xgb_model_12", xgb_model_12),
#     ("scv_model_12", scv_model_12),
# ]

<IPython.core.display.Javascript object>

# 3.2. 5 features

In [15]:
rf_model_5 = RandomForestClassifier(
    random_state=randomseed,
    n_estimators=100,
    max_depth=12,
    min_samples_split=2,
    min_samples_leaf=10,
    max_features="auto",
)

xgb_model_5 = xgb.XGBClassifier(objective="binary:logistic", random_state=randomseed)

scv_model_5 = svmgpu(
    C=70,
    cache_size=200,
    class_weight={},
    coef0=0.0,
    decision_function_shape="ovr",
    degree=3,
    gamma=0.001,
    gpu_id=0,
    kernel="linear",
    max_iter=-1,
    max_mem_size=-1,
    n_jobs=-1,
    probability=True,
    random_state=42,
    shrinking=True,
    tol=0.001,
    verbose=False,
)


scv_model_cpu_5 = SVC(
    C=70,
    cache_size=200,
    class_weight={},
    coef0=0.0,
    decision_function_shape="ovr",
    degree=3,
    gamma=0.001,
    #     gpu_id=0,
    kernel="linear",
    max_iter=-1,
    #     max_mem_size=-1,
    #     n_jobs=-1,
    probability=True,
    random_state=42,
    shrinking=True,
    tol=0.001,
    verbose=False,
)


weakmodles.append(rf_model_5)
weakmodles.append(xgb_model_5)
weakmodles.append(scv_model_5)

estimators.append(("rf_model_5", rf_model_5))
estimators.append(("xgb_model_5", xgb_model_5))
estimators.append(("scv_model_5", scv_model_cpu_5))


<IPython.core.display.Javascript object>

# 3.3. 10 Features

In [16]:
top10colscomb = [
    (0, 1, 2, 3, 4, 5, 6, 9, 10, 11),
    (0, 1, 3, 4, 6, 7, 8, 9, 10, 11),
    (0, 1, 2, 3, 4, 5, 6, 7, 10, 11),
    (0, 1, 2, 3, 4, 5, 6, 8, 10, 11),
    (0, 1, 2, 3, 5, 6, 8, 9, 10, 11),
    (0, 1, 3, 4, 5, 6, 8, 9, 10, 11),
    (0, 1, 3, 4, 5, 6, 7, 9, 10, 11),
    (0, 1, 2, 3, 4, 5, 8, 9, 10, 11),
    (0, 1, 2, 3, 4, 5, 6, 7, 9, 11),
    (0, 1, 2, 3, 4, 6, 7, 9, 10, 11),
]

# modles used in the 12 features set
rf_model_10 = RandomForestClassifier(
    bootstrap=True,
    class_weight=None,
    criterion="gini",
    max_depth=12,
    max_features="auto",
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    min_samples_leaf=10,
    min_samples_split=2,
    min_weight_fraction_leaf=0.0,
    n_estimators=100,
    n_jobs=None,
    oob_score=False,
    random_state=42,
    verbose=0,
    warm_start=False,
)

xgb_model_10 = xgb.XGBClassifier(
    base_score=0.5,
    booster="gbtree",
    colsample_bylevel=1,
    colsample_bynode=1,
    colsample_bytree=1,
    gamma=0,
    learning_rate=0.1,
    max_delta_step=0,
    max_depth=3,
    min_child_weight=1,
    missing=None,
    n_estimators=100,
    n_jobs=1,
    nthread=None,
    objective="multi:softprob",
    random_state=42,
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    seed=None,
    silent=None,
    subsample=1,
    verbosity=1,
)

scv_model_10 = svmgpu(
    C=1000,
    cache_size=100,
    class_weight={},
    coef0=0.0,
    decision_function_shape="ovo",
    degree=3,
    gamma=0.1,
    gpu_id=0,
    kernel="linear",
    max_iter=-1,
    max_mem_size=-1,
    n_jobs=-1,
    probability=False,
    random_state=None,
    shrinking=False,
    tol=0.001,
    verbose=False,
)

scv_model_cpu_10 = SVC(
    C=1000,
    cache_size=100,
    class_weight={},
    coef0=0.0,
    decision_function_shape="ovo",
    degree=3,
    gamma=0.1,  # gpu_id=0,
    kernel="linear",
    max_iter=-1,  # max_mem_size=-1, n_jobs=-1,
    probability=True,
    random_state=None,
    shrinking=False,
    tol=0.001,
    verbose=False,
)

# generate the models based on the selected columns list and the ml classifiers
for i in range(5):
    weakmodles.append(make_pipeline(ColumnSelector(cols=top10colscomb[i]), rf_model_10))
    weakmodles.append(
        make_pipeline(ColumnSelector(cols=top10colscomb[i]), xgb_model_10)
    )
    weakmodles.append(
        make_pipeline(ColumnSelector(cols=top10colscomb[i]), scv_model_10)
    )

for i in range(5):
    estimators.append(
        (
            (
                "rf_model_10" + str(i),
                (make_pipeline(ColumnSelector(cols=top10colscomb[i]), rf_model_10)),
            )
        )
    )
    estimators.append(
        (
            "xgb_model_10" + str(i),
            (make_pipeline(ColumnSelector(cols=top10colscomb[i]), xgb_model_10)),
        )
    )
    estimators.append(
        (
            "scv_model_cpu_10" + str(i),
            (make_pipeline(ColumnSelector(cols=top10colscomb[i]), scv_model_cpu_10)),
        )
    )

<IPython.core.display.Javascript object>

# 3.4. 9 Features

In [17]:
topcols9comb = [
    (0, 1, 2, 3, 5, 6, 8, 10, 11),
    (0, 1, 2, 3, 4, 6, 9, 10, 11),
    (0, 1, 2, 3, 4, 5, 6, 9, 11),
    (0, 1, 3, 4, 6, 7, 9, 10, 11),
    (0, 1, 3, 4, 5, 8, 9, 10, 11),
    (0, 1, 3, 4, 5, 6, 9, 10, 11),
    (0, 1, 3, 4, 5, 6, 8, 10, 11),
    (0, 1, 3, 4, 6, 7, 8, 10, 11),
    (0, 1, 3, 4, 5, 6, 7, 8, 11),
    (0, 1, 3, 5, 6, 8, 9, 10, 11),
]

# modles used in the 12 features set
rf_model_9 = RandomForestClassifier(
    bootstrap=True,
    class_weight=None,
    criterion="gini",
    max_depth=12,
    max_features="auto",
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    min_samples_leaf=10,
    min_samples_split=2,
    min_weight_fraction_leaf=0.0,
    n_estimators=100,
    n_jobs=None,
    oob_score=False,
    random_state=randomseed,
    verbose=0,
    warm_start=False,
)

xgb_model_9 = xgb.XGBClassifier(
    base_score=0.5,
    booster="gbtree",
    colsample_bylevel=1,
    colsample_bynode=1,
    colsample_bytree=1,
    gamma=0,
    learning_rate=0.1,
    max_delta_step=0,
    max_depth=3,
    min_child_weight=1,
    missing=None,
    n_estimators=100,
    n_jobs=1,
    nthread=None,
    objective="multi:softprob",
    random_state=randomseed,
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    seed=None,
    silent=None,
    subsample=1,
    verbosity=1,
)

scv_model_9 = svmgpu(
    C=1000,
    cache_size=100,
    class_weight={},
    coef0=0.0,
    decision_function_shape="ovo",
    degree=3,
    gamma=0.1,
    gpu_id=0,
    kernel="linear",
    max_iter=-1,
    max_mem_size=-1,
    n_jobs=-1,
    probability=False,
    random_state=randomseed,
    shrinking=False,
    tol=0.001,
    verbose=False,
)

scv_model_cpu_9 = SVC(
    C=1000,
    cache_size=100,
    class_weight={},
    coef0=0.0,
    decision_function_shape="ovo",
    degree=3,
    gamma=0.1,  # gpu_id=0,
    kernel="linear",
    max_iter=-1,  # max_mem_size=-1, n_jobs=-1,
    probability=True,
    random_state=randomseed,
    shrinking=False,
    tol=0.001,
    verbose=False,
)


# generate the models based on the selected columns list and the ml classifiers
for i in range(5):
    weakmodles.append(make_pipeline(ColumnSelector(cols=topcols9comb[i]), rf_model_9))
    weakmodles.append(make_pipeline(ColumnSelector(cols=topcols9comb[i]), xgb_model_9))
    weakmodles.append(make_pipeline(ColumnSelector(cols=topcols9comb[i]), scv_model_9))

for i in range(5):
    estimators.append(
        (
            (
                "rf_model_9" + str(i),
                (make_pipeline(ColumnSelector(cols=topcols9comb[i]), rf_model_9)),
            )
        )
    )
    estimators.append(
        (
            "xgb_model_9" + str(i),
            (make_pipeline(ColumnSelector(cols=topcols9comb[i]), xgb_model_9)),
        )
    )
    estimators.append(
        (
            "scv_model_cpu_9" + str(i),
            (make_pipeline(ColumnSelector(cols=topcols9comb[i]), scv_model_cpu_9)),
        )
    )

<IPython.core.display.Javascript object>

# 3.5. 8 Features

In [18]:
top8colscomb = [
    (0, 1, 2, 3, 5, 6, 10, 11),
    (0, 1, 2, 3, 4, 6, 10, 11),
    (0, 1, 2, 3, 4, 9, 10, 11),
    (0, 1, 2, 3, 5, 9, 10, 11),
    (0, 1, 3, 4, 5, 6, 9, 11),
    (0, 1, 3, 4, 6, 9, 10, 11),
    (0, 1, 3, 4, 8, 9, 10, 11),
    (0, 1, 2, 6, 8, 9, 10, 11),
    (0, 1, 2, 3, 5, 8, 10, 11),
    (0, 1, 3, 4, 5, 6, 10, 11),
]

# modles used in the 12 features set
rf_model_8 = RandomForestClassifier(
    bootstrap=True,
    class_weight=None,
    criterion="gini",
    max_depth=12,
    max_features="auto",
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    min_samples_leaf=10,
    min_samples_split=2,
    min_weight_fraction_leaf=0.0,
    n_estimators=100,
    n_jobs=None,
    oob_score=False,
    random_state=randomseed,
    verbose=0,
    warm_start=False,
)

xgb_model_8 = xgb.XGBClassifier(
    base_score=0.5,
    booster="gbtree",
    colsample_bylevel=1,
    colsample_bynode=1,
    colsample_bytree=1,
    gamma=0,
    learning_rate=0.1,
    max_delta_step=0,
    max_depth=3,
    min_child_weight=1,
    missing=None,
    n_estimators=100,
    n_jobs=1,
    nthread=None,
    objective="multi:softprob",
    random_state=randomseed,
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    seed=None,
    silent=None,
    subsample=1,
    verbosity=1,
)

scv_model_8 = svmgpu(
    C=1000,
    cache_size=100,
    class_weight={},
    coef0=0.0,
    decision_function_shape="ovo",
    degree=3,
    gamma=0.1,
    gpu_id=0,
    kernel="linear",
    max_iter=-1,
    max_mem_size=-1,
    n_jobs=-1,
    probability=False,
    random_state=None,
    shrinking=False,
    tol=0.001,
    verbose=False,
)

scv_model_cpu_8 = SVC(
    C=1000,
    cache_size=100,
    class_weight={},
    coef0=0.0,
    decision_function_shape="ovo",
    degree=3,
    gamma=0.1,  # gpu_id=0,
    kernel="linear",
    max_iter=-1,  # max_mem_size=-1, n_jobs=-1,
    probability=True,
    random_state=randomseed,
    shrinking=False,
    tol=0.001,
    verbose=False,
)

# generate the models based on the selected columns list and the ml classifiers
for i in range(5):
    weakmodles.append(make_pipeline(ColumnSelector(cols=top8colscomb[i]), rf_model_8))
    weakmodles.append(make_pipeline(ColumnSelector(cols=top8colscomb[i]), xgb_model_8))
    weakmodles.append(make_pipeline(ColumnSelector(cols=top8colscomb[i]), scv_model_8))

for i in range(5):
    estimators.append(
        (
            (
                "rf_model_8" + str(i),
                (make_pipeline(ColumnSelector(cols=top8colscomb[i]), rf_model_8)),
            )
        )
    )
    estimators.append(
        (
            "xgb_model_8" + str(i),
            (make_pipeline(ColumnSelector(cols=top8colscomb[i]), xgb_model_8)),
        )
    )
    estimators.append(
        (
            "scv_model_cpu_8" + str(i),
            (make_pipeline(ColumnSelector(cols=top8colscomb[i]), scv_model_cpu_8)),
        )
    )

<IPython.core.display.Javascript object>

# 3.6 7 Features

In [19]:
top7colscomb = [
    (0, 1, 3, 5, 6, 10, 11),
    (0, 1, 3, 4, 5, 10, 11),
    (0, 1, 2, 3, 4, 10, 11),
    (0, 1, 2, 3, 5, 10, 11),
    (0, 1, 3, 6, 8, 10, 11),
    (0, 1, 6, 8, 9, 10, 11),
    (0, 1, 6, 7, 8, 9, 10),
    (0, 1, 2, 3, 8, 10, 11),
    (0, 1, 2, 6, 8, 10, 11),
    (0, 1, 3, 8, 9, 10, 11),
]


rf_model_7 = RandomForestClassifier(
    bootstrap=True,
    class_weight=None,
    criterion="gini",
    max_depth=12,
    max_features="auto",
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    min_samples_leaf=10,
    min_samples_split=2,
    min_weight_fraction_leaf=0.0,
    n_estimators=100,
    n_jobs=None,
    oob_score=False,
    random_state=randomseed,
    verbose=0,
    warm_start=False,
)

xgb_model_7 = xgb.XGBClassifier(
    base_score=0.5,
    booster="gbtree",
    colsample_bylevel=1,
    colsample_bynode=1,
    colsample_bytree=1,
    gamma=0,
    learning_rate=0.1,
    max_delta_step=0,
    max_depth=3,
    min_child_weight=1,
    missing=None,
    n_estimators=100,
    n_jobs=1,
    nthread=None,
    objective="multi:softprob",
    random_state=randomseed,
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    seed=None,
    silent=None,
    subsample=1,
    verbosity=1,
)

scv_model_7 = svmgpu(
    C=1000,
    cache_size=100,
    class_weight={},
    coef0=0.0,
    decision_function_shape="ovo",
    degree=3,
    gamma=0.1,
    gpu_id=0,
    kernel="linear",
    max_iter=-1,
    max_mem_size=-1,
    n_jobs=-1,
    probability=False,
    random_state=randomseed,
    shrinking=False,
    tol=0.001,
    verbose=False,
)

scv_model_cpu_7 = SVC(
    C=1000,
    cache_size=100,
    class_weight={},
    coef0=0.0,
    decision_function_shape="ovo",
    degree=3,
    gamma=0.1,  # gpu_id=0,
    kernel="linear",
    max_iter=-1,  # max_mem_size=-1, n_jobs=-1,
    probability=True,
    random_state=randomseed,
    shrinking=False,
    tol=0.001,
    verbose=False,
)


# generate the models based on the selected columns list and the ml classifiers
for i in range(5):
    weakmodles.append(make_pipeline(ColumnSelector(cols=top7colscomb[i]), rf_model_7))
    weakmodles.append(make_pipeline(ColumnSelector(cols=top7colscomb[i]), xgb_model_7))
    weakmodles.append(make_pipeline(ColumnSelector(cols=top7colscomb[i]), scv_model_7))

for i in range(5):
    estimators.append(
        (
            (
                "rf_model_7" + str(i),
                (make_pipeline(ColumnSelector(cols=top7colscomb[i]), rf_model_7)),
            )
        )
    )
    estimators.append(
        (
            "xgb_model_7" + str(i),
            (make_pipeline(ColumnSelector(cols=top7colscomb[i]), xgb_model_7)),
        )
    )
    estimators.append(
        (
            "scv_model_cpu_7" + str(i),
            (make_pipeline(ColumnSelector(cols=top7colscomb[i]), scv_model_cpu_7)),
        )
    )

<IPython.core.display.Javascript object>

# 3.7. 6 Features

In [20]:
top6colscomb = [
    (0, 1, 8, 9, 10),
    (0, 1, 6, 8, 10),
    (0, 1, 7, 9, 10),
    (0, 1, 2, 8, 10),
    (0, 1, 5, 8, 10),
    (0, 1, 5, 6, 11),
    (0, 1, 7, 8, 10),
    (0, 1, 9, 10, 11),
    (0, 1, 5, 10, 11),
    (0, 1, 2, 5, 11),
]

# modles used in the 12 features set
rf_model_6 = RandomForestClassifier(
    bootstrap=True,
    class_weight=None,
    criterion="gini",
    max_depth=12,
    max_features="auto",
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    min_samples_leaf=10,
    min_samples_split=2,
    min_weight_fraction_leaf=0.0,
    n_estimators=100,
    n_jobs=None,
    oob_score=False,
    random_state=randomseed,
    verbose=0,
    warm_start=False,
)

xgb_model_6 = xgb.XGBClassifier(
    base_score=0.5,
    booster="gbtree",
    colsample_bylevel=1,
    colsample_bynode=1,
    colsample_bytree=1,
    gamma=0,
    learning_rate=0.1,
    max_delta_step=0,
    max_depth=3,
    min_child_weight=1,
    missing=None,
    n_estimators=100,
    n_jobs=1,
    nthread=None,
    objective="multi:softprob",
    random_state=randomseed,
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    seed=None,
    silent=None,
    subsample=1,
    verbosity=1,
)

scv_model_6 = svmgpu(
    C=1000,
    cache_size=100,
    class_weight={},
    coef0=0.0,
    decision_function_shape="ovo",
    degree=3,
    gamma=0.1,
    gpu_id=0,
    kernel="linear",
    max_iter=-1,
    max_mem_size=-1,
    n_jobs=-1,
    probability=False,
    random_state=randomseed,
    shrinking=False,
    tol=0.001,
    verbose=False,
)

scv_model_cpu_6 = SVC(
    C=1000,
    cache_size=100,
    class_weight={},
    coef0=0.0,
    decision_function_shape="ovo",
    degree=3,
    gamma=0.1,  # gpu_id=0,
    kernel="linear",
    max_iter=-1,  # max_mem_size=-1, n_jobs=-1,
    probability=True,
    random_state=randomseed,
    shrinking=False,
    tol=0.001,
    verbose=False,
)


# generate the models based on the selected columns list and the ml classifiers
for i in range(5):
    weakmodles.append(make_pipeline(ColumnSelector(cols=top6colscomb[i]), rf_model_6))
    weakmodles.append(make_pipeline(ColumnSelector(cols=top6colscomb[i]), xgb_model_6))
    weakmodles.append(make_pipeline(ColumnSelector(cols=top6colscomb[i]), scv_model_6))

for i in range(5):
    estimators.append(
        (
            (
                "rf_model_6" + str(i),
                (make_pipeline(ColumnSelector(cols=top7colscomb[i]), rf_model_6)),
            )
        )
    )
    estimators.append(
        (
            "xgb_model_6" + str(i),
            (make_pipeline(ColumnSelector(cols=top7colscomb[i]), xgb_model_6)),
        )
    )
    estimators.append(
        (
            "scv_model_cpu_6" + str(i),
            (make_pipeline(ColumnSelector(cols=top7colscomb[i]), scv_model_cpu_6)),
        )
    )

<IPython.core.display.Javascript object>

# 3.8. 5 Features

In [21]:
top5colscomb = [
    (0, 1, 8, 9, 10),
    (0, 1, 6, 8, 10),
    (0, 1, 7, 9, 10),
    (0, 1, 2, 8, 10),
    (0, 1, 5, 8, 10),
    (0, 1, 5, 6, 11),
    (0, 1, 7, 8, 10),
    (0, 1, 9, 10, 11),
    (0, 1, 5, 10, 11),
    (0, 1, 2, 5, 11),
]


rf_model_5_2 = RandomForestClassifier(
    bootstrap=True,
    class_weight=None,
    criterion="gini",
    max_depth=12,
    max_features="auto",
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    min_samples_leaf=10,
    min_samples_split=2,
    min_weight_fraction_leaf=0.0,
    n_estimators=100,
    n_jobs=None,
    oob_score=False,
    random_state=randomseed,
    verbose=0,
    warm_start=False,
)

xgb_model_5_2 = xgb.XGBClassifier(
    base_score=0.5,
    booster="gbtree",
    colsample_bylevel=1,
    colsample_bynode=1,
    colsample_bytree=1,
    gamma=0,
    learning_rate=0.1,
    max_delta_step=0,
    max_depth=3,
    min_child_weight=1,
    missing=None,
    n_estimators=100,
    n_jobs=1,
    nthread=None,
    objective="multi:softprob",
    random_state=randomseed,
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    seed=None,
    silent=None,
    subsample=1,
    verbosity=1,
)

scv_model_5_2 = svmgpu(
    C=1000,
    cache_size=100,
    class_weight={},
    coef0=0.0,
    decision_function_shape="ovo",
    degree=3,
    gamma=0.1,
    gpu_id=0,
    kernel="linear",
    max_iter=-1,
    max_mem_size=-1,
    n_jobs=-1,
    probability=False,
    random_state=randomseed,
    shrinking=False,
    tol=0.001,
    verbose=False,
)

scv_model_cpu_5_2 = SVC(
    C=1000,
    cache_size=100,
    class_weight={},
    coef0=0.0,
    decision_function_shape="ovo",
    degree=3,
    gamma=0.1,
    #     gpu_id=0,
    kernel="linear",
    max_iter=-1,
    # max_mem_size=-1,   n_jobs=-1,
    probability=True,
    random_state=randomseed,
    shrinking=False,
    tol=0.001,
    verbose=False,
)


# generate the models based on the selected columns list and the ml classifiers
for i in range(5):
    weakmodles.append(make_pipeline(ColumnSelector(cols=top5colscomb[i]), rf_model_5_2))
    weakmodles.append(
        make_pipeline(ColumnSelector(cols=top5colscomb[i]), xgb_model_5_2)
    )
    weakmodles.append(
        make_pipeline(ColumnSelector(cols=top5colscomb[i]), scv_model_5_2)
    )

for i in range(5):
    estimators.append(
        (
            (
                "rf_model_52" + str(i),
                (make_pipeline(ColumnSelector(cols=top5colscomb[i]), rf_model_5_2)),
            )
        )
    )
    estimators.append(
        (
            "xgb_model_52" + str(i),
            (make_pipeline(ColumnSelector(cols=top5colscomb[i]), xgb_model_5_2)),
        )
    )
    estimators.append(
        (
            "scv_model_cpu_52" + str(i),
            (make_pipeline(ColumnSelector(cols=top5colscomb[i]), scv_model_cpu_5_2)),
        )
    )

<IPython.core.display.Javascript object>

In [22]:
len(weakmodles)

96

<IPython.core.display.Javascript object>

In [23]:
len(estimators)

96

<IPython.core.display.Javascript object>

# 4. Generate Stacking Classifier

In [24]:
sclf = StackingClassifier(classifiers=weakmodles,verbose=2,
                           meta_classifier=RandomForestClassifier(n_estimators=500))


<IPython.core.display.Javascript object>

In [25]:
sclf.fit(xtrain, ytrain)

Fitting 96 classifiers...
Fitting classifier1: randomforestclassifier (1/96)
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=10, max_features='log2', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=12,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=42, verbose=1,
                       warm_start=False)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.4s finished


Fitting classifier2: xgbclassifier (2/96)
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)
Fitting classifier3: svc (3/96)
SVC(C=100, cache_size=100, class_weight={}, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma=0.1, gpu_id=0,
    kernel='linear', max_iter=-1, max_mem_size=-1, n_jobs=-1, probability=False,
    random_state=42, shrinking=False, tol=0.001, verbose=False)
Fitting classifier4: randomforestclassifier (4/96)
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=12, max_features='auto', max_

Fitting classifier14: pipeline (14/96)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=(0, 1, 2, 3, 4, 5, 6, 7, 10, 11),
                                drop_axis=False)),
                ('xgbclassifier',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, learning_rate=0.1,
                               max_delta_step=0, max_depth=3,
                               min_child_weight=1, missing=None,
                               n_estimators=100, n_jobs=1, nthread=None,
                               objective='multi:softprob', random_state=42,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                               seed=None, silent=None, subsample=1,
                               verbosity=1))],
         verbose=False)
Fitting classifier15: pipeline (15/96)


Fitting classifier24: pipeline (24/96)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=(0, 1, 2, 3, 5, 6, 8, 10, 11),
                                drop_axis=False)),
                ('svc',
                 SVC(C=1000, cache_size=100, class_weight={}, coef0=0.0,
                     decision_function_shape='ovo', degree=3, gamma=0.1,
                     gpu_id=0, kernel='linear', max_iter=-1, max_mem_size=-1,
                     n_jobs=-1, probability=False, random_state=42,
                     shrinking=False, tol=0.001, verbose=False))],
         verbose=False)
Fitting classifier25: pipeline (25/96)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=(0, 1, 2, 3, 4, 6, 9, 10, 11),
                                drop_axis=False)),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        crite

...

Fitting classifier84: pipeline (84/96)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=(0, 1, 8, 9, 10), drop_axis=False)),
                ('svc',
                 SVC(C=1000, cache_size=100, class_weight={}, coef0=0.0,
                     decision_function_shape='ovo', degree=3, gamma=0.1,
                     gpu_id=0, kernel='linear', max_iter=-1, max_mem_size=-1,
                     n_jobs=-1, probability=False, random_state=42,
                     shrinking=False, tol=0.001, verbose=False))],
         verbose=False)
Fitting classifier85: pipeline (85/96)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=(0, 1, 6, 8, 10), drop_axis=False)),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=12,
                                        max_features='auto',
  

Fitting classifier95: pipeline (95/96)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=(0, 1, 5, 8, 10), drop_axis=False)),
                ('xgbclassifier',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, learning_rate=0.1,
                               max_delta_step=0, max_depth=3,
                               min_child_weight=1, missing=None,
                               n_estimators=100, n_jobs=1, nthread=None,
                               objective='multi:softprob', random_state=42,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                               seed=None, silent=None, subsample=1,
                               verbosity=1))],
         verbose=False)
Fitting classifier96: pipeline (96/96)
Pipeline(memory=None,
         steps=[('columnse

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished


StackingClassifier(average_probas=False,
                   classifiers=[RandomForestClassifier(bootstrap=True,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=10,
                                                       max_features='log2',
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=2,
                                                       min_samples_split=12,
                                                       min_weight_fraction_leaf=0.0,
                                                       n_estimators=100,
                                                

<IPython.core.display.Javascript object>

In [26]:
ypred = sclf.predict((xtest))

print("accuracy \t", m.accuracy_score(ytest, ypred))
confmatrx = pd.DataFrame(
    m.confusion_matrix(ytest, ypred),
    columns=["Normal", "Prediabetes", "diabetes"],
    index=["Normal", "Prediabetes", "diabetes"],
)
confmatrx.head()

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished


accuracy 	 0.725


,Normal,Prediabetes,diabetes
Normal,145,55,0
Prediabetes,60,129,11
diabetes,3,36,161


<IPython.core.display.Javascript object>

In [27]:
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.70      0.72      0.71       200
           1       0.59      0.65      0.61       200
           2       0.94      0.81      0.87       200

    accuracy                           0.73       600
   macro avg       0.74      0.73      0.73       600
weighted avg       0.74      0.72      0.73       600



<IPython.core.display.Javascript object>

# Compute the performance of each model of the sclf

In [28]:
# for i in range(len(sclf.clfs_)):
#     ypred = sclf.clfs_[i].predict((xtest))
#     print(type(sclf.clfs_[i]), "accuracy \t", m.accuracy_score(ytest, ypred))

<IPython.core.display.Javascript object>

In [29]:
stackedmodels = sclf.clfs_
for i in range(len(stackedmodels)):
    ypred = stackedmodels[i].predict(xtest)
    print(type(sclf.clfs_[i]), "accuracy \t", m.accuracy_score(ytest, ypred))
    print("Accuracy = ", m.accuracy_score(ytest, ypred))

    confmatrx = pd.DataFrame(
        m.confusion_matrix(ytest, ypred),
        columns=["Normal", "Prediabetes", "diabetes"],
        index=["Normal", "Prediabetes", "diabetes"],
    )
    print(confmatrx.head())

    print(m.classification_report(ytest, ypred))

    print(i)

<class 'sklearn.ensemble.forest.RandomForestClassifier'> accuracy 	 0.73
Accuracy =  0.73
             Normal  Prediabetes  diabetes
Normal          150           49         1
Prediabetes      56          124        20
diabetes          3           33       164
              precision    recall  f1-score   support

           0       0.72      0.75      0.73       200
           1       0.60      0.62      0.61       200
           2       0.89      0.82      0.85       200

    accuracy                           0.73       600
   macro avg       0.74      0.73      0.73       600
weighted avg       0.74      0.73      0.73       600

0
<class 'xgboost.sklearn.XGBClassifier'> accuracy 	 0.735
Accuracy =  0.735
             Normal  Prediabetes  diabetes
Normal          151           48         1
Prediabetes      59          116        25
diabetes          4           22       174
              precision    recall  f1-score   support

           0       0.71      0.76      0.73       200

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished


<class 'thundersvm.thundersvm.SVC'> accuracy 	 0.735
Accuracy =  0.735
             Normal  Prediabetes  diabetes
Normal          150           49         1
Prediabetes      55          119        26
diabetes          4           24       172
              precision    recall  f1-score   support

           0       0.72      0.75      0.73       200
           1       0.62      0.59      0.61       200
           2       0.86      0.86      0.86       200

    accuracy                           0.73       600
   macro avg       0.73      0.73      0.73       600
weighted avg       0.73      0.73      0.73       600

2
<class 'sklearn.ensemble.forest.RandomForestClassifier'> accuracy 	 0.7333333333333333
Accuracy =  0.7333333333333333
             Normal  Prediabetes  diabetes
Normal          148           51         1
Prediabetes      55          127        18
diabetes          4           31       165
              precision    recall  f1-score   support

           0       0.71      

             Normal  Prediabetes  diabetes
Normal          153           47         0
Prediabetes      55          119        26
diabetes          3           23       174
              precision    recall  f1-score   support

           0       0.73      0.77      0.74       200
           1       0.63      0.59      0.61       200
           2       0.87      0.87      0.87       200

    accuracy                           0.74       600
   macro avg       0.74      0.74      0.74       600
weighted avg       0.74      0.74      0.74       600

16
<class 'sklearn.pipeline.Pipeline'> accuracy 	 0.7383333333333333
Accuracy =  0.7383333333333333
             Normal  Prediabetes  diabetes
Normal          150           49         1
Prediabetes      55          119        26
diabetes          4           22       174
              precision    recall  f1-score   support

           0       0.72      0.75      0.73       200
           1       0.63      0.59      0.61       200
           2

             Normal  Prediabetes  diabetes
Normal          148           52         0
Prediabetes      57          126        17
diabetes          4           32       164
              precision    recall  f1-score   support

           0       0.71      0.74      0.72       200
           1       0.60      0.63      0.61       200
           2       0.91      0.82      0.86       200

    accuracy                           0.73       600
   macro avg       0.74      0.73      0.73       600
weighted avg       0.74      0.73      0.73       600

30
<class 'sklearn.pipeline.Pipeline'> accuracy 	 0.7466666666666667
Accuracy =  0.7466666666666667
             Normal  Prediabetes  diabetes
Normal          150           49         1
Prediabetes      54          123        23
diabetes          4           21       175
              precision    recall  f1-score   support

           0       0.72      0.75      0.74       200
           1       0.64      0.61      0.63       200
           2

Accuracy =  0.73
             Normal  Prediabetes  diabetes
Normal          148           52         0
Prediabetes      59          125        16
diabetes          4           31       165
              precision    recall  f1-score   support

           0       0.70      0.74      0.72       200
           1       0.60      0.62      0.61       200
           2       0.91      0.82      0.87       200

    accuracy                           0.73       600
   macro avg       0.74      0.73      0.73       600
weighted avg       0.74      0.73      0.73       600

45
<class 'sklearn.pipeline.Pipeline'> accuracy 	 0.7483333333333333
Accuracy =  0.7483333333333333
             Normal  Prediabetes  diabetes
Normal          153           47         0
Prediabetes      54          121        25
diabetes          4           21       175
              precision    recall  f1-score   support

           0       0.73      0.77      0.74       200
           1       0.64      0.60      0.62      

<class 'sklearn.pipeline.Pipeline'> accuracy 	 0.7416666666666667
Accuracy =  0.7416666666666667
             Normal  Prediabetes  diabetes
Normal          150           49         1
Prediabetes      53          121        26
diabetes          4           22       174
              precision    recall  f1-score   support

           0       0.72      0.75      0.74       200
           1       0.63      0.60      0.62       200
           2       0.87      0.87      0.87       200

    accuracy                           0.74       600
   macro avg       0.74      0.74      0.74       600
weighted avg       0.74      0.74      0.74       600

59
<class 'sklearn.pipeline.Pipeline'> accuracy 	 0.7316666666666667
Accuracy =  0.7316666666666667
             Normal  Prediabetes  diabetes
Normal          147           52         1
Prediabetes      55          127        18
diabetes          4           31       165
              precision    recall  f1-score   support

           0       0.71

              precision    recall  f1-score   support

           0       0.72      0.75      0.74       200
           1       0.62      0.61      0.62       200
           2       0.88      0.85      0.87       200

    accuracy                           0.74       600
   macro avg       0.74      0.74      0.74       600
weighted avg       0.74      0.74      0.74       600

73
<class 'sklearn.pipeline.Pipeline'> accuracy 	 0.7433333333333333
Accuracy =  0.7433333333333333
             Normal  Prediabetes  diabetes
Normal          150           47         3
Prediabetes      51          119        30
diabetes          3           20       177
              precision    recall  f1-score   support

           0       0.74      0.75      0.74       200
           1       0.64      0.59      0.62       200
           2       0.84      0.89      0.86       200

    accuracy                           0.74       600
   macro avg       0.74      0.74      0.74       600
weighted avg       0.

             Normal  Prediabetes  diabetes
Normal          151           49         0
Prediabetes      53          123        24
diabetes          3           26       171
              precision    recall  f1-score   support

           0       0.73      0.76      0.74       200
           1       0.62      0.61      0.62       200
           2       0.88      0.85      0.87       200

    accuracy                           0.74       600
   macro avg       0.74      0.74      0.74       600
weighted avg       0.74      0.74      0.74       600

91
<class 'sklearn.pipeline.Pipeline'> accuracy 	 0.58
Accuracy =  0.58
             Normal  Prediabetes  diabetes
Normal          152            7        41
Prediabetes      53           18       129
diabetes          3           19       178
              precision    recall  f1-score   support

           0       0.73      0.76      0.75       200
           1       0.41      0.09      0.15       200
           2       0.51      0.89      0

<IPython.core.display.Javascript object>

# 5. Generate Votting Classifer

In [30]:
votingclf = VotingClassifier(estimators=estimators, voting="hard")

<IPython.core.display.Javascript object>

In [ ]:
votingclf.fit(xtrain, ytrain)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.0s finished


In [ ]:
ypred = votingclf.predict(xtest.astype(float))

print("accuracy \t", m.accuracy_score(ytest, ypred))
confmatrx = pd.DataFrame(
    m.confusion_matrix(ytest, ypred),
    columns=["Normal", "Prediabetes", "diabetes"],
    index=["Normal", "Prediabetes", "diabetes"],
)
confmatrx.head()

In [ ]:
print(classification_report(ytest, ypred))

In [ ]:
stackedmodels = votingclf.estimators_
for i in range(len(stackedmodels)):
    ypred = stackedmodels[i].predict(xtest)
    print(type(sclf.clfs_[i]), "accuracy \t", m.accuracy_score(ytest, ypred))
    print("Accuracy = ", m.accuracy_score(ytest, ypred))

    confmatrx = pd.DataFrame(
        m.confusion_matrix(ytest, ypred),
        columns=["Normal", "Prediabetes", "diabetes"],
        index=["Normal", "Prediabetes", "diabetes"],
    )
    print(confmatrx.head())

    print(m.classification_report(ytest, ypred))

    print(i)

# 6  Generate Votting Classifer soft voting

In [ ]:

scv_model_cpu_withProb = SVC(
  C=100,
    cache_size=100,
    class_weight={},
    coef0=0.0,
    decision_function_shape="ovo",
    degree=3,
    gamma=0.1,
#     gpu_id=0,
    kernel="linear",
    max_iter=-1,
#     max_mem_size=-1,
#     n_jobs=-1,
    probability=True,
    random_state=randomseed,
    shrinking=False,
    tol=0.001,
    verbose=False,
)


In [ ]:
votingclf2 = VotingClassifier(
    estimators=[
        ("rf_model", rf_model),
        ("xgb_model", xgb_model),
        ("scv_model", scv_model_cpu_withProb),
    ],
    voting="soft",
)

In [ ]:
votingclf2.fit(xtrain, ytrain)

In [ ]:
ypred = votingclf2.predict(xtest.astype(float))

print("accuracy \t", m.accuracy_score(ytest, ypred))
confmatrx = pd.DataFrame(
    m.confusion_matrix(ytest, ypred),
    columns=["Normal", "Prediabetes", "diabetes"],
    index=["Normal", "Prediabetes", "diabetes"],
)
confmatrx.head()

In [ ]:
print(classification_report(ytest, ypred))

In [ ]:
stackedmodels = votingclf2.estimators_
for i in range(len(stackedmodels)):
    ypred = stackedmodels[i].predict(xtest)
    print(type(sclf.clfs_[i]), "accuracy \t", m.accuracy_score(ytest, ypred))
    print("Accuracy = ", m.accuracy_score(ytest, ypred))

    confmatrx = pd.DataFrame(
        m.confusion_matrix(ytest, ypred),
        columns=["Normal", "Prediabetes", "diabetes"],
        index=["Normal", "Prediabetes", "diabetes"],
    )
    print(confmatrx.head())

    print(m.classification_report(ytest, ypred))

    print(i)